In [1]:
import os
import sys
import pandas as pd
import numpy as np
import requests
from datetime import date, datetime, tzinfo, timedelta
sys.path.insert(0, os.path.abspath('../Utilities'))

In [2]:
import liminal_data
import liminal_eod

In [10]:
api_key = '62108f321e00c9.86864641'

In [4]:
#dir(liminal_data)

In [5]:
years_back =1
days_back = int(years_back*365.25)
end = datetime.today().strftime("%Y-%m-%d")
start = (datetime.today()-timedelta(days=days_back)).strftime("%Y-%m-%d")
code = 'MSFT.US'
timezone = 'US/Eastern'

In [6]:
# Test Function

In [7]:
msft = liminal_eod.get_daily_bars(code,start,end,'sa',timezone)
msft.dtypes

Open                  float64
High                  float64
Low                   float64
Close                 float64
Volume                  int64
Engine                 object
Source                 object
Update    datetime64[ns, UTC]
dtype: object

In [8]:
msft.index[0].tz

<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>

In [9]:
#msft['UpdateString'] = msft['Update'].dt.strftime(liminal_data.liminal_format)

In [10]:
#dt = msft['Update'][0]

In [11]:
#dt2 = dt.strftime(liminal_data.liminal_format)

In [12]:
# save dataframe to s3
bucket = 'lc-market-data'
folder = 'Equities/US/'
filename = 'MSFT_test.txt'

#liminal_data.save_s3_object(bucket,folder,filename,msft)
liminal_data.save_s3_bars(bucket,folder,filename,msft)


Saved S3 Object: lc-market-data:Equities/US/MSFT_test.txt


In [13]:
pulledMSFT = liminal_data.get_s3_bars(bucket,folder,filename)

In [14]:
pulledMSFT.dtypes

Open                  float64
High                  float64
Low                   float64
Close                 float64
Volume                  int64
Engine                 object
Source                 object
Update    datetime64[ns, UTC]
dtype: object

In [15]:
#pulledMSFT.index = pd.to_datetime(pulledMSFT.index,format='%Y-%m-%d %H:%M:%S%z')

In [16]:
# S3 Pulled dat has pytz.fixedOffset
pulledMSFT.index[0].tz


pytz.FixedOffset(-300)

In [17]:
# EOD Data has tz aware timezone
msft.index[0].tz

<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>

In [18]:
result = pd.concat([msft, pulledMSFT], axis=1).reindex(msft.index)

ValueError: Tz-aware datetime.datetime cannot be converted to datetime64 unless utc=True

In [ ]:
# Modify DataFrame
# Change Index Name
msft.index.names = ['DateIndex']
# Generate Date String
msft['DateString'] = msft.index.strftime(liminal_data.liminal_format)
# Create New Column 
msft = msft.assign(Date = lambda x:(x['DateString'].str[:-2]+':'+x['DateString'].str[-2:]))
# reorder columns
msft = msft[['Date','Open','High','Low','Close','Volume','Engine','Source','Update']]
# Drop Index
msft = msft.reset_index(drop=True)

In [ ]:
msft.head()

In [ ]:
datestr =msft['DateString'][0]

In [ ]:
newDs =  "{0}:{1}".format(datestr[:-2],datestr[-2:]) 
newDs

In [ ]:
#msft.apply(lambda x: x['DateString'],axis=1)
msft = msft.assign(Dstr = lambda x:(x['DateString'].str[:-2]+':'+x['DateString'].str[-2:]))

In [ ]:
msft['Dstr'][0]

In [ ]:
## Pull data
url = f'https://eodhistoricaldata.com/api/eod/{code}?from={start}&to={end}&api_token={api_key}'

## Convert string to datetime 
# Attempt 1 - parse_dates - Failed
#df = pd.read_csv(url,parse_dates=['Date'])

# Fails becasue last row is bad

# Attempt 2 - parse dates with custom parser
# 1. Pull data
df = pd.read_csv(url)


In [ ]:
#2 . DropNA
df = df.dropna(subset=['Open', 'High','Low','Close'])

In [ ]:
#3. Convert to date time
df['Date'] = pd.to_datetime(df['Date'], format ='%Y-%m-%d')
eodDate = '%Y-%'

In [ ]:
df.loc[0,'Date'].day_name()

In [ ]:
# set index
df.set_index('Date', inplace= True)

In [ ]:
df.tail()

In [ ]:
# test slice

In [ ]:
df['2022-01']

In [ ]:
# check date time time zone
df.index.dtype

In [ ]:
# localize to TIme zone
df = df.tz_localize(tz='US/Eastern')

In [ ]:
df.index

In [ ]:
#conver to other 

In [ ]:



print(startDate)

print(endDate)

timezone = 'America/New_York'
#eoddf = liminal_eod.get_daily_bars_tz(code,startDate, endDate,'sa',timezone)
eoddf = liminal_eod.get_daily_bars(code,startDate, endDate,'sa')


#aapl = extract_historical_csvdata2('AAPL', startDate,endDate)[:-1]
#aapl.rename(columns={'Open':'Open_EOD','High':'High_EOD','Low':'Low_EOD','Close':'Close_EOD','Volume':'Volume_EOD'},inplace=True)
#aapl.sort_values(by='Date', ascending=True, kind='quicksort', inplace = True)
#aapl.shape

In [ ]:
import pytz
#eoddf = eoddf.index.tz_localize("US/Eastern")
#eoddf.tz_localize('UTC', level=0)
print(eoddf.index.dtype)


In [ ]:
eoddf

In [ ]:
eoddf.dtypes

In [ ]:
eoddf.index = eoddf.index.apply(eoddf.replace(tzinfo=None))

#eoddf = eoddf.tz_localize('UTC')
#my_timezone = pytz.timezone('Europe/Berlin')
#eoddf.index = eoddf.index.dt.tz_convert(my_timezone)

In [ ]:
splits = liminal_eod.get_splits(code,startDate,endDate)
divs

In [ ]:
index = 'STOXX50E.INDX'
ndx = liminal_eod.get_index_constituants(index)
ndx

In [ ]:
comdf = pd.concat([eoddf,df],axis=1)

In [ ]:
comdf.shape

In [ ]:
combinedrange = comdf.loc['2022-01-20']
combinedrange

In [ ]:
comdf['Diff'] = comdf['Close_EOD']-comdf['Close']
comdf['DiffP'] = comdf['Diff'] /comdf['Close']

In [ ]:
comdf['DiffP'].plot()

In [ ]:
plt.plot(comdf.index,comdf['Close'])
plt.plot(comdf.index,comdf['Close_EOD'])
plt.show()

In [ ]:
def extract_sa_daily(symbol, start_date, end_date):
    #function [required] – splitadjusted.
    url = f'https://eodhistoricaldata.com/api/technical/{symbol}?from={start_date}&to={end_date}&api_token={api_key}&function=splitadjusted'
    df = pd.read_csv(url).set_index('Date')
    return df

aapl = extract_split_adjusted('AAPL', '2011-01-01', '2021-10-01')[:-1]
aapl.head()

In [ ]:
def get_index_constituants(index):
    url = f'https://eodhistoricaldata.com/api/fundamentals/{index}.INDX?api_token={api_key}'
    data = requests.get(url).json()
    components = data['Components']
    cdf = pd.DataFrame(components)
    cdf = cdf.transpose()
    return cdf

def get_index_json(index):
    url = f'https://eodhistoricaldata.com/api/fundamentals/{index}.INDX?api_token={api_key}'
    data = requests.get(url).json()
    return data


code = 'N225'
indexList = get_index_constituants(code)
indexList

#indexjson = get_index_json(code)
#indexjson

In [ ]:
def get_etf_constituants(etfCode):
    url = f'https://eodhistoricaldata.com/api/fundamentals/{etfCode}?api_token={api_key}'
    data = requests.get(url).json()
    components = data['ETF_Data']['Holdings']
    cdf = pd.DataFrame(components)
    cdf = cdf.transpose()
    return cdf

def get_etf_json(index):
    url = f'https://eodhistoricaldata.com/api/fundamentals/{index}?api_token={api_key}'
    data = requests.get(url).json()
    return data


code = 'EWJ.US'
indexList = get_etf_constituants(code)
indexList

#indexjson = get_etf_json(code)
#indexjson['ETF_Data']['Holdings']

In [ ]:
def get_divi_json(code,start):
    url= f'https://eodhistoricaldata.com/api/div/{code}?api_token={api_key}&from={start}&fmt=json'
    data = requests.get(url).json()
    return data

divs = get_divi_json('MSFT','2000-01-01')
ddf = pd.DataFrame(divs)
ddf

In [ ]:
def get_splits_json(code,start):
    url= f'https://eodhistoricaldata.com/api/splits/{code}?api_token={api_key}&from={start}&fmt=json'
    data = requests.get(url).json()
    return data

divs = get_splits_json('FB','2000-01-01')
ddf = pd.DataFrame(divs)
ddf

In [ ]:
cdf

In [13]:
def extract_fundamentals(symbol):
    url = f'https://eodhistoricaldata.com/api/fundamentals/{symbol}?api_token={api_key}' 
    data = requests.get(url).json()
    return data


def extract_exchange_details(exCode):
    url =f'https://eodhistoricaldata.com/api/exchange-details/{exCode}?api_token={api_key}'
    data = requests.get(url).json()
    return data

#tsla_fundamentals = extract_fundamentals('TSLA')
#tsla_fundamentals

tse = extract_exchange_details('TSE')
tse

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import eod_interface

In [ ]:
import os
import sys

print(sys.path)